The goal of this project is to use the stack data structure to implement an algorithm that can evaluate complex numerical expressions.

When we write an expression, we use infix notation, meaning that we put the operators between the two operands. For example 1 + 2 is in infix notation because the + operator is between the operands 1 and 2.

For a computer, it's much easier to evaluate an expression written in postfix notation. In postfix notation, the operands appear before the operator. The infix expression 1 + 2 becomes 1 2 + in postfix notation.

We want to implement a function that evaluate that can evaluate expressions stored in string utilizing postfix expressions

## Impliment Stack and LinkedList

In [3]:
class Node:
    
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None

class LinkedList:

    def __init__(self):
        self.head = None
        self.tail = None
        self.length = 0

    def append(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.tail.next = new_node
            new_node.prev = self.tail
            self.tail = new_node
        self.length += 1

    def __iter__(self):
        self._iter_node = self.head
        return self 

    def __next__(self):
        if self._iter_node is None:
            raise StopIteration
        ret = self._iter_node.data
        self._iter_node = self._iter_node.next
        return ret

    def prepend(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
        self.length += 1

    def __len__(self):
        return self.length

    def __str__(self):
        return str([value for value in self])

class Stack(LinkedList):
    def push(self, data):
        self.append(data)
        
    def peek(self):
        return self.tail.data
    
    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None           
        self.length -= 1
        return ret            

In [3]:
# given a string, function tokenize() return a list of strings with 
# the tokens in the expression  
def tokenize(string):
    return string.split()
string = "12 2 4 + / 21 *"
tokenize(string)

['12', '2', '4', '+', '/', '21', '*']

## Functions to process operators in postfix evaluation

In [9]:
#  function that processes the - (minus) token:
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)
    
#  function that processes the + (plus) token:
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop() 
    result = top + second_to_top
    stack.push(result)

#  function that processes the * (times) token:
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop() 
    result = top * second_to_top
    stack.push(result)

#  function that processes the / (divide) token:
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop() 
    result = second_to_top / top
    stack.push(result)
    
#  function that processes the ** (power) token:
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop() 
    result = second_to_top ** top
    stack.push(result)

## Evaluating postfix expressions Algorithm

We can now implement an algorithm to evaluate an expression in postfix notation. To do so, we need to do the following:

1. Tokenize the expression using the tokenize() function
2. Initialize an empty stack
3. For each token, do the following:
    * If the token is an operator, call the corresponding function to process it. For example, if we find a +, we call the process_plus() function.
    * Otherwise (the token is a number) we push that number to the top of the stack. Since each token is a string, we'll need to convert it to a float first.
4. Return the value that is left in the stack.

In [21]:
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    for token in tokens:
        if token == '+':
            process_plus(stack)
        elif token == '-':
            process_minus(stack)
        elif token == '*':
            process_times(stack)
        elif token == '/':
            process_divide(stack)
        elif token == '**':
            process_pow(stack)
        else:
            stack.push(float(token))
    return stack.pop()     

In [22]:
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]
for expression in expressions:
     print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


## Processing tokens in infix to postfix conversions

We will convert an expression from infix to postfix by  implement the Shunting-yard algorithm. In the Shunting-yard algorithm, we'll need to compare the precedence of the operators. Thus first, we the precedence dictionary to compare the precedence of two operators, then handling parenthesis, operations priority, number then finally the algorithm itself.

### Precedence Dictionary 

In [29]:
precedence = {"+" : 1, "-" : 1, "*" : 2, "/" : 2, "**" : 3}

print(precedence["+"] < precedence["*"])
print(precedence["+"] < precedence["-"])
print(precedence["/"] < precedence["**"])

True
False
True


### Handling parenthesis

Handling open parenthesis "(" by:
*    Push the "(" token into the stack for later use when we find a closing parenthesis. 

In [24]:
def process_opening_parenthesis(stack):
        stack.push('(')

Handling close parenthesis ")"  by:
1. While the top of the stack isn't an opening parenthesis, (, pop the top element, and append it to the postfix token list.    
2. Pop the opening parentheses out of the stack at the end.

In [25]:
def process_closing_parenthesis(stack, postfix):
    # Add tokens until we find the open bracket
    while stack.peek() != '(':
        element = stack.pop()
        postfix.append(element)
    # Remove the opening bracket
    stack.pop()

### Handing Operators priority

Operator + , - , * , / or  **:
* While the top of the stack is also an operator with a precedence greater than or equal to this operator, pop the top element and append it to the postfix token list.
* Push the current operator to the top of the stack.

In [27]:
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        element = stack.pop()
        postfix.append(element)
    stack.push(operator)

### Operand number

Operand (any number):
* Push the token into the postfix token list.

In [28]:
def process_number(postfix, number):
    postfix.append(number)

### The Shunting-yard Algorithm

The Shunting-yard Algorithm can be implemented as follow:
1. We start by splitting the expression into tokens using the tokenize() function.
2. We initialize an empty stack.
3. We initialize an empty postfix token list.
4. Iterate over all tokens, and for each, do the following:
    * If the token is "(", we call the process_opening_parenthesis() function.
    * If the token is ")", we call the process_closing_parenthesis() function.
    * If the token is an operator, we call the process_operator() function.
    * Otherwise, the token is a number, and we call the process_number() function.
5. After processing all tokens, we use a while loop to pop the remaining stack element into the postfix token list.
6. Use the str.join() method to convert the postfix token list into a string.   

In [33]:
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    operators = ['+', '-', '*', '/', '**']
    for token in tokens:
        if token == '(':
            process_opening_parenthesis(stack)
        elif token == ')':
            process_closing_parenthesis(stack, postfix)
        elif token in operators:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    while len(stack) > 0:
        postfix.append(stack.pop())
    return " ".join(postfix)            

We now have a function that can transform an infix expression into postfix notation and a function that can evaluate an expression in postfix notation. By combining the two, we can write a function named evaluate() that returns the value of an expression in infix notation.

In [34]:
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

Let's test our function 

In [35]:
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for expression in expressions:
    print(evaluate(expression))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0
